In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"

from dualneuron.screening.sets import ImagenetImages, RenderedImages
from dualneuron.screening.run import activations

from pathlib import Path
import dualneuron
import numpy as np

import matplotlib.pyplot as plt

token = os.getenv("HF_TOKEN")
data_dir = os.getenv("DATA_DIR")

In [2]:
data = np.load(data_dir + "dryad/v4_imagenet_ordered_responses.npz")
indices = np.load(data_dir + "dryad/v4_imagenet_ordered_indices.npz")

model_name = "V4ColorTaskDriven"
package_dir = Path(dualneuron.__file__).parent
mask_path = package_dir / "twins" / model_name / "mask.npy"
mask = np.load(mask_path)

In [ ]:
from scipy import stats
from tqdm import tqdm

skewness = {}

for key in tqdm(data.keys()):
    activations = data[key]
    skew = stats.skew(activations)
    skewness[key] = skew

sorted_neurons = sorted(
    skewness.items(), 
    key=lambda x: x[1], 
    reverse=True
)

print("Neurons ordered from most sparse to least sparse:")
print("-" * 50)
for neuron_id, skew in sorted_neurons[:10]:  # Show top 10
    print(f"Neuron: {neuron_id}, Skewness: {skew:.4f}")

ordered_neurons = [neuron_id for neuron_id, _ in sorted_neurons]

In [ ]:
highpoles = {}
num_imgs = 10

for key in tqdm(ordered_neurons):
    highpoles[key] = indices[key][-num_imgs:]

In [ ]:
dset = ImagenetImages(
    data_dir=data_dir + "datasets",
    token=token,
    split='train',
    use_center_crop=True,
    use_resize_output=True,
    use_grayscale=False,
    use_normalize=False,
    use_mask=True,
    use_norm=False,
    mask=mask,
    num_channels=3,
    output_size=(224, 224),
    crop_size=236,
    bg_value=0.45,
    norm=None,
)

In [ ]:
highpole_images = {}

for key in tqdm(highpoles.keys()):
    image_indices = highpoles[key]
    images = []
    for idx in image_indices:
        img, _ = dset[idx]
        images.append(img.permute(1, 2, 0))

    highpole_images[key] = np.stack(images) 

In [ ]:
tail = 40
fig, axes = plt.subplots(num_imgs, tail, figsize=(0.99 * tail, num_imgs))

for i, neuron_id in enumerate(ordered_neurons[:tail]):
    images = highpole_images[neuron_id]
    skew = skewness[neuron_id]
    act = data[neuron_id][-1]

    for j, img in enumerate(images[:num_imgs]):
        if j == 0:
            axes[j, i].set_title(f"{skew:.2f} \n {act:.2f}", fontsize=8)
        axes[j, i].imshow(img)
        axes[j, i].axis('off')

plt.subplots_adjust(wspace=0., hspace=0.)
plt.show()

fig, axes = plt.subplots(num_imgs, tail, figsize=(0.99 * tail, num_imgs))

for i, neuron_id in enumerate(ordered_neurons[-tail:]):
    images = highpole_images[neuron_id]
    skew = skewness[neuron_id]
    act = data[neuron_id][-1]

    for j, img in enumerate(images[:num_imgs]):
        if j == 0:
            axes[j, i].set_title(f"{skew:.2f} \n {act:.2f}", fontsize=8)
        axes[j, i].imshow(img)
        axes[j, i].axis('off')

plt.subplots_adjust(wspace=0., hspace=0.)
plt.show()